In [2]:
!pip install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os

In [4]:
!pip install hyperopt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import h2o
from h2o.estimators import H2OXGBoostEstimator
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample

In [6]:
import sys
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [9]:
### Starting H2o Session
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp9tc4hxc9
  JVM stdout: /tmp/tmp9tc4hxc9/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp9tc4hxc9/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,"14 days, 11 hours and 5 minutes"
H2O_cluster_name:,H2O_from_python_unknownUser_qjd17w
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.486 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [10]:
#Train = pd.read_csv("Train.csv")
#Test = pd.read_csv("Test.csv")
TTD= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Mono_Data/3_Auto_Ttd_Binned_Data_260324.csv')
TTD.shape
# oot_binned = pd.read_csv("/home/username/aniruddha/NewBinned_BureauLN_BP_oot_mono_60_18m.csv")

(685215, 144)

In [11]:
TTD['Race'].value_counts()

Race
white       386048
missing     173486
black        60934
hispanic     55029
api           8771
aian           816
multiple       128
others           3
Name: count, dtype: int64

In [12]:
TTD["Race_black"] = np.where(TTD["Race"] == "black",1, 0)
TTD["Race_hispanic"] = np.where(TTD["Race"] == "hispanic",1, 0)
TTD["Race_api"] = np.where((TTD["Race"] == "asian")|(TTD["Race"] == "api"),1, 0)
TTD["age_cuts_flag"] = np.where(TTD["age_cuts"] == "Age 62 and over",1, 0)
TTD["Gender_flag"] = np.where(TTD["Gender"] == "F",1, 0)

In [13]:
TTD["Race_api"].value_counts()

Race_api
0    676444
1      8771
Name: count, dtype: int64

In [14]:
TTD['client'].value_counts(dropna= False)

client
Arkansas    147800
Tower       137007
Numerica    128156
Guardian     82074
GESA         80506
SECU         54132
Bethpage     32968
HFCU         14129
Hawaii        5973
NCU           2470
Name: count, dtype: int64

In [15]:
### Convert the Variable into categorical
var_to_cat= '''businesstitleleadership
proflictypecategory
addrcurrentdeedmailing
addrcurrentphoneservice
addrcurrentsubjectowned
addrinputdeedmailing
addrinputphoneservice
addrinputproblems
addrinputsubjectowned
addrlastmoveecontrajectory
addronfilecollege
addronfilecorrectional
addronfilehighrisk
addrprevioussubjectowned
alertregulatorycondition
assetownership
assetpersonal
assetprop
assetpropnewestmortgagetype
bankruptcychapter
bankruptcystatus
businessassociation
educationattendance
educationevidence
educationinstitutionrating
educationprogramattended
inquiryauto12month
inquirybanking12month
inquirycollections12month
inquirynonshortterm12month
inquiryshortterm12month
inquirytelcom12month
shorttermloanrequest
shorttermloanrequest12month
shorttermloanrequest24month
subjectnewestrecord12month
sourcevoterregistration'''.splitlines()
len(var_to_cat)

37

In [16]:
for var in var_to_cat:
    TTD[var]= TTD[var].astype(str)

In [17]:
TTD.columns

Index(['unique_id', 'client', 'app_date', 'approve_flag', 'funded_flag',
       'fico_pb', 'fico_cb', 'bad_flag', 'product', 'auto_score',
       ...
       'shorttermloanrequest24month', 'sourcecredheadertimenewest',
       'sourcevoterregistration', 'subjectnewestrecord12month',
       'subjectrecordtimenewest', 'Race_black', 'Race_hispanic', 'Race_api',
       'age_cuts_flag', 'Gender_flag'],
      dtype='object', length=149)

In [18]:
from sklearn.model_selection import train_test_split

TTD_train,TTD_test = train_test_split(TTD, test_size=0.10,random_state=4127)

TTD_train['approve_flag'].mean(), TTD_test['approve_flag'].mean()

(0.5861133497542538, 0.5859869822830623)

In [19]:
req_cols= ['assetpropevercount', 'sourcenonderogcount06month', 'addronfilecount', 'addrinputcountyratio', 'assetpropnewestsaleprice', 'addrcurrenttaxvalue', 'addrinputavmvalue', 'addrinputavmvalue60month', 'addrinputlastsaleprice', 'addrinputtaxmarketvalue', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'assetpropcurrenttaxtotal', 'addrcurrentavmvalue', 'assetproppurchasetimeoldest', 'addrpreviouslengthofres', 'addrinputlengthofres', 'addrcurrenttaxmarketvalue', 'addrprevioustimenewest', 'addrcurrenttractratio', 'sourcenonderogcount12month', 'assetpropcurrentcount', 'addrinputsubjectcount', 'sourcenonderogcount', 'addrinputtaxvalue', 'assetpropeversoldcount', 'businessassociationtimeoldest', 'addrinputtractratio', 'derogtimenewest', 'addrchangecount24month', 'addrinputtimelastsale', 'educationinstitutionprivate', 'addrinputblockratio', 'businesstitleleadership', 'addrcurrentlengthofres', 'addrcurrentavmratio12monthprior', 'addrcurrentblockratio', 'derogcount', 'profliccount', 'addrinputavmratio60monthprior', 'addrcurrentcountyratio', 'assetpropsaletimeoldest', 'subjectrecordtimeoldest', 'addrcurrenttimenewest', 'proflictypecategory', 'criminalnonfelonytimenewest', 'lienjudgmentdollartotal', 'addrcurrentlastsalesprice', 'addrinputtimenewest', 'evictiontimenewest', 'bankruptcycount', 'addrcurrenttimelastsale', 'addrinputtimeoldest', 'addrlastmovetaxratiodiff', 'criminalnonfelonycount', 'sourcenonderogcount03month', 'addrchangecount03month', 'addrchangecount06month', 'addrchangecount12month', 'addrchangecount60month', 'addrcurrentavmratio60monthprior', 'addrcurrentavmvalue12month', 'addrcurrentavmvalue60month', 'addrcurrentcorrectional', 'addrcurrentdeedmailing', 'addrcurrentphoneservice', 'addrcurrentsubjectowned', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'addrinputavmvalue12month', 'addrinputdeedmailing', 'addrinputphoneservice', 'addrinputproblems', 'addrinputsubjectowned', 'addrlastmoveecontrajectory', 'addronfilecollege', 'addronfilecorrectional', 'addronfilehighrisk', 'addrpreviouscorrectional', 'addrprevioussubjectowned', 'addrprevioustimeoldest', 'alertregulatorycondition', 'assetownership', 'assetpersonal', 'assetpersonalcount', 'assetprop', 'assetpropnewestmortgagetype', 'assetproppurchasecount12month', 'assetproppurchasetimenewest', 'assetpropsalepurchaseratio', 'assetpropsaletimenewest', 'assetpropsoldcount12month', 'bankruptcychapter', 'bankruptcycount24month', 'bankruptcydismissed24month', 'bankruptcystatus', 'bankruptcytimenewest', 'businessassociation', 'criminalfelonycount', 'criminalfelonycount12month', 'criminalfelonytimenewest', 'criminalnonfelonycount12month', 'derogcount12month', 'educationattendance', 'educationevidence', 'educationinstitutionrating', 'educationprogramattended', 'evictioncount', 'evictioncount12month', 'inquiryauto12month', 'inquirybanking12month', 'inquirycollections12month', 'inquirynonshortterm12month', 'inquiryshortterm12month', 'inquirytelcom12month', 'lienjudgmentcount', 'lienjudgmentcount12month', 'lienjudgmentcourtcount', 'lienjudgmentforeclosurecount', 'lienjudgmentothercount', 'lienjudgmentsmallclaimscount', 'lienjudgmenttaxcount', 'lienjudgmenttimenewest', 'shorttermloanrequest', 'shorttermloanrequest12month', 'shorttermloanrequest24month', 'sourcecredheadertimenewest', 'sourcevoterregistration', 'subjectnewestrecord12month', 'subjectrecordtimenewest']
len(req_cols)

130

In [20]:
TTD_train.shape, TTD_test.shape

((616693, 149), (68522, 149))

In [21]:
TTD['bankruptcystatus'].value_counts(dropna= False)

bankruptcystatus
0       648409
1        33203
2         3175
[-1]       428
Name: count, dtype: int64

In [22]:
TTD['bankruptcystatus'].dtypes

dtype('O')

In [23]:
ttd_train_h2o = h2o.H2OFrame(TTD_train)
ttd_test_h2o = h2o.H2OFrame(TTD_test)
print(ttd_train_h2o.shape,ttd_test_h2o.shape)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
(616693, 149) (68522, 149)


In [24]:
ttd_train_h2o['bankruptcystatus'].unique()

C1
nan
0
2
1


In [26]:
ttd_train_h2o['bankruptcystatus'].asfactor().unique()

C1
0
1
2


In [27]:
for col in var_to_cat:
    ttd_train_h2o[col] = ttd_train_h2o[col].asfactor()
    ttd_test_h2o[col] = ttd_test_h2o[col].asfactor()

In [28]:
for k in TTD['bankruptcychapter'].unique():
    print(k, ttd_train_h2o[ttd_train_h2o['bankruptcychapter']== k].shape)

0 (583449, 149)
1 (25452, 149)
2 (7771, 149)
[-1] (0, 149)


In [29]:
path_to_save= '/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Protected_Class_Data/'

In [30]:
def recall_cal(data_h2o,data,name,dep):
    predict = xgbm_FL.predict(data_h2o)
    p =predict.as_data_frame()
    dd = data.loc[:,unique_cols + [dep_var]]
    dd = pd.DataFrame(dd)
    dd = dd.reset_index()
    dd['RECWGT'] = 1
    dd['conwt'] = 1
    pred = pd.concat([dd.loc[:,(dep_var,"RECWGT","conwt")],p],axis=1)
    dep_flag_rate = (pred["RECWGT"]*pred[dep_var]).sum()/pred["RECWGT"].sum()
    pred = pred.sort_values(by=["p1"],ascending=False).reset_index()
    pred["cum_wgt"] = pred["RECWGT"].cumsum()
    pred["cum_wgt_nor"] = pred["cum_wgt"]/pred["RECWGT"].sum()
    pred["scorecut"] = np.where(pred["cum_wgt_nor"]<=dep_flag_rate,1,0)
    pred["TP"] = np.where((pred["scorecut"]==1) & (pred[dep_var]==1),1,0)
    recall = pred["TP"].sum()/pred[dep_var].sum()
    print( pred["TP"].sum(),pred[dep_var].sum())
    pred.to_csv(path_to_save+name+dep_var +"_dataset.csv")
    return recall, dep_flag_rate

In [31]:
unique_cols = ['unique_id', 'client']
wt_col = "wt2"
folds = 10
predictors = ttd_train_h2o.columns
req_cols = req_cols# config.dropna().Feature_Name.to_list()
model_cols = unique_cols + req_cols
ttd_train_h2o=ttd_train_h2o[model_cols+['Race_black','Race_hispanic','Race_api','age_cuts_flag','Gender_flag']]#,'mmct_tag']]


print(ttd_train_h2o.shape)
print(len(req_cols))

(616693, 137)
130


In [ ]:
%%time
#dep_var = "Race_black"
data_recall = pd.DataFrame()
for dep_var in ['Race_black','Race_hispanic','Race_api','age_cuts_flag','Gender_flag']:
    print(dep_var)

    xgbm_FL = H2OXGBoostEstimator(
        ntrees=2000,    learn_rate=0.01,
        max_depth=3,    model_id=dep_var,
        distribution = "bernoulli",    seed=42,
        nfolds = 0,    ignored_columns = unique_cols)
    model_cols = unique_cols + [dep_var]+ req_cols
    ttd_train_h2o[dep_var] = ttd_train_h2o[dep_var].asfactor()
    xgbm_FL.train(y=dep_var, training_frame=ttd_train_h2o[model_cols])
    xgbm_FL.download_mojo(path = path_to_save+dep_var+".zip")
    recall,dep_flag_rate = recall_cal(ttd_train_h2o,TTD_train,"Train",dep_var)
    print(recall, dep_flag_rate)
    recall1,dep_flag_rate1 = recall_cal(ttd_test_h2o,TTD_test,"Test",dep_var)
    print(recall1,dep_flag_rate1)
    var_imp = xgbm_FL._model_json['output']['variable_importances'].as_data_frame()
    var_imp['Var_unique'] = var_imp['variable'].str.split('.', expand=True)[0]
    var_imp.to_csv(path_to_save+dep_var +"_var_im.csv")
    df = pd.DataFrame([[dep_var,recall,recall1,dep_flag_rate,dep_flag_rate1]], columns=['Dep', 'recall',"test_recall","dep_flag_rate","dep_flag_rate1"])
    data_recall = pd.concat([data_recall, df])#data_recall.append(df)
    data_recall.to_csv(path_to_save +"recall.csv")

Race_black
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
19868 54863
0.3621384175127135 0.08896322805674785
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
2226 6071
0.3666611760830176 0.088599281982429
Race_hispanic
xgboost Model Build progress: |█████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
15721 49529
0.3174100022209211 0.08031386767808293
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
1687 5500
0.30672727272727274 0.08026619187998016
Race_api
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
1205 786

H2OConnectionError: Unexpected HTTP error: HTTPConnectionPool(host='localhost', port=54321): Max retries exceeded with url: /3/Jobs/$03017f00000132d4ffffffff$_a368970a75a47066bb89c8d5ca704d16 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f1c245aace0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [29]:
df

,Dep,recall,test_recall,dep_flag_rate,dep_flag_rate1
0,age_cuts_flag,0.689225,0.687966,0.109834,0.111453


In [30]:
data_recall

,Dep,recall,test_recall,dep_flag_rate,dep_flag_rate1
0,Race_black,0.362138,0.366661,0.088963,0.088599
0,Race_hispanic,0.317410,0.306727,0.080314,0.080266
0,Race_api,0.153269,0.150715,0.012749,0.013266
0,age_cuts_flag,0.689225,0.687966,0.109834,0.111453


In [31]:
# predict = xgbm_black.predict(ttd_test_h2o)
# p =predict.as_data_frame()
# dd = TTD_test.loc[:,unique_cols + [dep_var]]
# dd = pd.DataFrame(dd)
# dd = dd.reset_index()
# dd['RECWGT'] = 1
# dd['conwt'] = 1
# pred = pd.concat([dd.loc[:,(dep_var,"RECWGT","conwt")],p],axis=1)
# print(pred["RECWGT"].sum(),((pred["RECWGT"]*pred[dep_var]).sum()),pred["RECWGT"].max())
# dep_flag_rate = (pred["RECWGT"]*pred[dep_var]).sum()/pred["RECWGT"].sum()
# pred = pred.sort_values(by=["p1"],ascending=False).reset_index()
# pred["cum_wgt"] = pred["RECWGT"].cumsum()
# pred["cum_wgt_nor"] = pred["cum_wgt"]/pred["RECWGT"].sum()
# pred["scorecut"] = np.where(pred["cum_wgt_nor"]<=dep_flag_rate,1,0)
# pred["TP"] = np.where((pred["scorecut"]==1) & (pred[dep_var]==1),1,0)
# recall_test = pred["TP"].sum()/pred[dep_var].sum()
# print(recall_test) 

In [32]:
# var_imp = xgbm_black._model_json['output']['variable_importances'].as_data_frame()
# var_imp['Var_unique'] = var_imp['variable'].str.split('.', expand=True)[0]
# var_imp.head(2)

In [33]:
# var_imp.to_csv(path_to_save+dep_var +"_var_im.csv")

In [23]:
data_recall= pd.read_csv(path_to_save +"recall.csv")
data_recall

,Unnamed: 0,Dep,recall,test_recall,dep_flag_rate,dep_flag_rate1
0,0,Race_black,0.362138,0.366661,0.088963,0.088599
1,0,Race_hispanic,0.317410,0.306727,0.080314,0.080266
2,0,Race_api,0.153269,0.150715,0.012749,0.013266
3,0,age_cuts_flag,0.689225,0.687966,0.109834,0.111453


In [ ]:
%%time
#dep_var = "Race_black"
# data_recall = pd.DataFrame()
for dep_var in ['Gender_flag']:#['Race_black','Race_hispanic','Race_api','age_cuts_flag','Gender_flag']:
    print(dep_var)

    xgbm_FL = H2OXGBoostEstimator(
        ntrees=2000,    learn_rate=0.01,
        max_depth=3,    model_id=dep_var,
        distribution = "bernoulli",    seed=42,
        nfolds = 0,    ignored_columns = unique_cols)
    model_cols = unique_cols + [dep_var]+ req_cols
    ttd_train_h2o[dep_var] = ttd_train_h2o[dep_var].asfactor()
    xgbm_FL.train(y=dep_var, training_frame=ttd_train_h2o[model_cols])
    xgbm_FL.download_mojo(path = path_to_save+dep_var+".zip")
    recall,dep_flag_rate = recall_cal(ttd_train_h2o,TTD_train,"Train",dep_var)
    print(recall, dep_flag_rate)
    recall1,dep_flag_rate1 = recall_cal(ttd_test_h2o,TTD_test,"Test",dep_var)
    print(recall1,dep_flag_rate1)
    var_imp = xgbm_FL._model_json['output']['variable_importances'].as_data_frame()
    var_imp['Var_unique'] = var_imp['variable'].str.split('.', expand=True)[0]
    var_imp.to_csv(path_to_save+dep_var +"_var_im.csv")
    df = pd.DataFrame([[dep_var,recall,recall1,dep_flag_rate,dep_flag_rate1]], columns=['Dep', 'recall',"test_recall","dep_flag_rate","dep_flag_rate1"])
    data_recall = pd.concat([data_recall, df])#data_recall.append(df)
    data_recall.to_csv(path_to_save +"recall.csv")

Gender_flag
xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
117041 234238
0.4996670053535293 0.3798291856726118
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
13122 26023
0.5042462437074895 0.3797758384168588
CPU times: user 20.8 s, sys: 1.14 s, total: 21.9 s
Wall time: 1h 3min 16s


In [25]:
data_recall

,Unnamed: 0,Dep,recall,test_recall,dep_flag_rate,dep_flag_rate1
0,0.0,Race_black,0.362138,0.366661,0.088963,0.088599
1,0.0,Race_hispanic,0.317410,0.306727,0.080314,0.080266
2,0.0,Race_api,0.153269,0.150715,0.012749,0.013266
3,0.0,age_cuts_flag,0.689225,0.687966,0.109834,0.111453
0,NaN,Gender_flag,0.499667,0.504246,0.379829,0.379776


In [32]:
TTD['Race_black'].sum(), len(TTD)

(60934, 685215)

In [33]:
TTD['Race_hispanic'].sum(), len(TTD)

(55029, 685215)

In [34]:
TTD['Race_api'].sum(), len(TTD)

(8771, 685215)

In [35]:
TTD['age_cuts_flag'].sum(), len(TTD)

(75371, 685215)

In [36]:
TTD['Gender_flag'].sum(), len(TTD)

(260261, 685215)

In [7]:
train= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Mono_Data/3_Auto_Train_Binned_Data_260324.csv')
test= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Mono_Data/3_Auto_Test_Binned_Data_260324.csv')
oot= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Mono_Data/3_Auto_Oot_Binned_Data_260324.csv')
ttd= pd.read_csv('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/Without_LDA_Auto_LN_V2/Mono_Data/3_Auto_Ttd_Binned_Data_260324.csv')
train.shape, test.shape, oot.shape

((146855, 145), (62938, 145), (37121, 144))

In [8]:
ttd.shape

(685215, 144)

In [5]:
train.groupby('client')['fico_pb'].apply(lambda x: x.isnull().sum())

client
Arkansas      0
Bethpage     84
CPM           0
GESA          3
Guardian    100
HFCU          7
Hawaii        0
NCU           0
Numerica      0
SECU          0
Tower         0
Name: fico_pb, dtype: int64

In [6]:
test.groupby('client')['fico_pb'].apply(lambda x: x.isnull().sum())

client
Arkansas     0
Bethpage    30
CPM          0
GESA         1
Guardian    45
HFCU         0
Hawaii       0
NCU          0
Numerica     0
SECU         0
Tower        0
Name: fico_pb, dtype: int64

In [7]:
oot.groupby('client')['fico_pb'].apply(lambda x: x.isnull().sum())

client
Arkansas     0
Bethpage    18
CPM          0
GESA         2
Guardian    21
HFCU         0
Hawaii       0
NCU          0
Numerica     0
SECU         0
Tower        0
Name: fico_pb, dtype: int64

In [10]:
ttd.groupby('client')['fico_pb'].apply(lambda x: x.isnull().sum())

client
Arkansas       0
Bethpage     972
GESA         368
Guardian    4606
HFCU         178
Hawaii         0
NCU            0
Numerica     102
SECU           0
Tower          0
Name: fico_pb, dtype: int64

In [11]:
ttd[ttd['funded_flag']==1].groupby('client')['fico_pb'].apply(lambda x: x.isnull().sum())

client
Arkansas      0
Bethpage    114
GESA          4
Guardian    145
HFCU          7
Hawaii        0
NCU           0
Numerica      0
SECU          0
Tower         0
Name: fico_pb, dtype: int64

In [6]:
pd.Series(['addrinputcountyratio', "addrinputavmvalue12month","addrinputavmvalue60month","addrcurrentavmvalue12month","addrinputavmvalue","sourcevoterregistration","addrinputavmratio60monthprior","addrcurrentcountyratio","addrcurrentphoneservice","addrinputsubjectcount","proflictypecategory","assetpersonal","educationevidence"]).reset_index()

,index,0
0,0,addrinputcountyratio
1,1,addrinputavmvalue12month
2,2,addrinputavmvalue60month
3,3,addrcurrentavmvalue12month
4,4,addrinputavmvalue
5,5,sourcevoterregistration
6,6,addrinputavmratio60monthprior
7,7,addrcurrentcountyratio
8,8,addrcurrentphoneservice
9,9,addrinputsubjectcount
